# More Customization with Message Passing APIs

In previous sessions, we have learned using the built-in [graph convolution modules](https://docs.dgl.ai/api/python/nn.pytorch.html#module-dgl.nn.pytorch.conv) and the message passing API to build a multi-layer graph neural network. This session provides 
However, sometimes one desires to invent a new way of aggregating neighbor information. DGL's message passing APIs are designed for this scenario.

In this tutorial, you will learn:
* DGL's message passing APIs.
* Design a new graph convolution module.

## More customization

In DGL, we provide many built-in message and reduce functions under the `dgl.function` package.

![api](../asset/dgl-mp.png)

You can find more details in [the API doc](https://docs.dgl.ai/api/python/function.html).

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

These APIs allow one to quickly implement new graph convolution modules.

## Graph convolutioin with edge weights

For example, the following example implements a new graph convolution that aggregates neighbor representations using a weighted average.

$$
h_{\mathcal{N}(v)}^l\leftarrow \text{AGGREGATE}_l\{h_u^{l-1} * e_{uv},\forall u\in\mathcal{N}(v)\}
$$

$$
h_v^l\leftarrow \sigma\left(W^l\cdot \text{CONCAT}(h_v^{l-1}, h_{\mathcal{N}(v)}^l) \right)
$$


Note that `edata` member can hold edge features which can also take part in message passing.

In [ ]:
class GraphConvWithEdgeWeight(nn.Module):
    """Graph convolution module using edge weights.
    
    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(GraphConvWithEdgeWeight, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)
    
    def forward(self, g, h, w):
        """Forward computation
        
        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        w : Tensor
            The edge weight.
        """
        with g.local_scope():
            g.ndata['h'] = h
            g.edata['w'] = w
            # update_all is a message passing API.
            g.update_all(fn.u_mul_e('h', 'w', 'm'), fn.mean('m', 'h_neigh'))
            h_neigh = g.ndata['h_neigh']
            h_total = torch.cat([h, h_neigh], dim=1)
            return F.relu(self.linear(h_total))

## Even more customization by user-defined function

DGL allows user-defined message and reduce function for the maximal expressiveness. Here is a user-defined message function that is equivalent to `fn.u_mul_e('h', 'w', 'm')`.

In [ ]:
def u_mul_e_udf(edges):
    return {'m' : edges.src['h'] * edges.data['w']}

`edges` has three members: `src`, `data` and `dst`, representing the source node feature, edge feature, and destination node feature for all edges.

## Recap

* Using the built-in message and reduce functions in `dgl.function` to customize a new NN module.
* User-defined function provides even more flexibility.